In [1]:
import tifffile
import python_test
import pandas as pd
import numpy as np
import skimage
from scipy import ndimage
from cellpose import models
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")


In [3]:
force_redo = False
masks_path = '/home/sim/OneDrive/Data/by_projects/gcamp/masks_new_trained_scratch/'
tif_path = '/home/sim/OneDrive/Data/by_projects/gcamp/tif/'
save_path = '/home/sim/OneDrive/Data/by_projects/gcamp/output_npy_saved/'

In [16]:
all_masks = os.listdir(masks_path)
create_folder(save_path)
random_masks = np.random.choice(all_masks)
print(random_masks)

Folder '/home/sim/OneDrive/Data/by_projects/gcamp/output_npy_saved/' already exists.
2 APB 2023 03 08_46_7.tif


In [17]:
save_name = random_masks[:-4] + '.npy'
if save_name in os.listdir(save_path) and force_redo == False:
    print('CSV for ' + random_masks + ' already exists.')
else:
    X = tifffile.imread(tif_path + random_masks[:-4] + '.tif')
    Y = tifffile.imread(masks_path + random_masks)
    index = Y.shape[0]
    X = X[:index,:,:]

    df = do_it(X, Y)


Get intensities


/home/sim/anaconda3/envs/cuda_118/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/sim/anaconda3/envs/cuda_118/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Get COM
Get border values
Do Crossvalidation
Save file


In [24]:
def filtered_masks(masks, common_cells):
    """Set all labels in masks not in common_cells to 0."""
    filter = np.isin(masks, common_cells)
    masks[~filter] = 0
    return masks

def extract_border_values(flt_masks, cell_lbl):
    def get_border_values(mask, cell_lbl):
        padded_mask = np.pad(mask, 1, "constant", constant_values=0) # Wherever there are no neighboring cells, we 
                                                                        # want zeroes, also on edges of image
        fltr = ndimage.binary_dilation(padded_mask==cell_lbl, structure=np.ones((3, 3)))
        bv = padded_mask[fltr]
        bv = bv[bv!=cell_lbl]
        return bv

    bv_count = 10
    bv_final = []
    for m in flt_masks:
        if not np.any(m==cell_lbl):
            continue
        bv = get_border_values(m,cell_lbl)
        if not np.any(bv==0):
            return bv
        q = len(bv[bv==0])/len(bv) # Check how much of the border is 0
        if q<0.05: # We're satisfied if 95% of the border is nonzero
            return bv
        if q<bv_count:
            bv_count = q
            bv_final = bv
    return bv_final

def get_ncc_for_bv(d, c, max_dt):
    bvs = np.array(d[c]['border_values'])
    bvs = bvs[bvs!=0]
    bvs_uniq = np.unique(bvs)
    ncc = 0
    dts = 0
    for bv in bvs_uniq:
        xcorr, dt = python_test.get_cc(np.array(d[c]['intensities']),
                        np.array(d[bv]['intensities']), max_dt)
        w = len(bvs[bvs==bv])/len(bvs[bvs!=0])
        ncc += np.max(xcorr)*w
        dts += dt[np.argmax(xcorr)]*w
    return ncc, dts

def get_max_freq(intensities, T):

    fft = np.fft.fft(intensities)
    freqs = np.fft.fftfreq(len(intensities), T)
    fft_filtered = fft*(freqs>(3/(3600))) #Remove all periods longer than 20 min  
        
    return np.abs(freqs[np.argmax(fft_filtered)])

def do_it2(X, masks, occurrence_limit=50, T=10, max_dt = 30):
    
    common_cells, counts = python_test.get_common_cells(masks, occurrence=occurrence_limit) # Get the cells to include in the intensity measurements
    fltr_masks = filtered_masks(masks, common_cells)
    d = {}

    for c in common_cells:
        print(c)
        # get intensities
        intensity = python_test.get_cell_intensities(c, masks, X, T=T)
        
        # get COMS
        coms = []
        for im in range(len(masks)):
            if np.any(masks[im]==c):
                coms.append(list(python_test.get_centers_of_mass(masks[im], c)[0]))
                break
        
        # get border values
        boarder_values = list(extract_border_values(fltr_masks, c))
        d[c] = {'intensities': intensity, 'com': coms, 'border_values': boarder_values}
    
    # get cross correlation
    for c in common_cells:
        d[c]['Weighted max NCC for border values'], d[c]['Weighted time difference at max NCC for border values'] = get_ncc_for_bv(d, c, max_dt)

        # get most prominent frequency
        d[c]['Most prominent frequency'] = get_max_freq(d[c]['intensities'], T)

    return d

d = do_it2(X, Y)

with open('dict_test.pkl', 'wb') as f:
    pickle.dump(d, f)



1
2
3
5
7
8
12
13
15
16
17
20
21
22
23
25
26
27
28
29
30
31
32
33
35
37
38
39
40
41
42
43
45
46
47
48
49
51
53
54
57
58
60
62
63
64
65
66
68
69
70
72
73
74
75
76
77


/home/sim/anaconda3/envs/cuda_118/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/sim/anaconda3/envs/cuda_118/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


78
80
81
82
85
88
89
90
91
92
93
95
96
97
99
100
116
117
135
157
197
198
201
205
206
207
210


In [25]:
with open('dict_test.pkl', 'wb') as f:
    pickle.dump(d, f)